In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Generics: Array-based queue

<div class="alert alert-block alert-success"> 
    Source code for this notebook can be found in the package <tt>ca.queensu.cisc124.notes.generics.basics</tt>.
</div>

We begin by implementing a fixed-size queue and then extend our implementation to
a queue that can grow in capacity by reallocating its internal array of elements.

The video of the dequeue operation in the [previous notebook](generics_queues.ipynb#vid-dequeue)
illustrates what happens in a physical queue when an element is dequeued: All of the
remaining elements in the queue get shifted forward one position. This also occurs
in the `ArrayList`-based implementation of a queue because removing the first element
of a list causes all of the remaining elements to be shifted forward one position 
in the list. This is also the reason why the dequeue operation has computational
complexity $O(n)$ in the `ArrayList`-based implementation.

We want to avoid shifting elements during the dequeue operation in our array-based
queue implementation. The trick to doing so is to keep track of where the front element
of the queue is stored in the array using a field named `front`. When a dequeue operation occurs, we obtain a
reference to the front element of the queue (so that we can return the element back
to the caller) and then move `front` one position to the right.

The enqueue operation is similar to the push operation for a stack. We use a field
named `back` to store the index of the element that is currently at the back of the queue.
When an enqueue operation occurs, we move `back` one position to the right and set
the array element at `back` to the newly enqueued element.

Similarly to our array-based stack implementation, we have a field named `size` that
keeps track of the number of elements in the queue, and a field named `arr` that
refers to the array used to store the elements of the queue.

| Field name | Purpose |
| :--- | :--- |
| `arr`   | the array used to store the elements of the queue  |
| `size`  | the number of elements currently in the queue |
| `front` | the array index of the element currently at the front of the queue |
| `back`  | the array index of the element currently at the back of the queue |

<div class="alert alert-block alert-warning">
    The use of the field <tt>back</tt> differs slightly from the back pointer shown
    in the videos in the <a href="./generics_queues.ipynb#notebook_id">previous notebook</a>
    where the back pointer pointed to the position after the last element of the queue.
    In this notebook, <tt>back</tt> is the index of the last element.
</div>

The array for a newly created empty queue of capacity 16 might look something like:

![A new empty queue](../resources/images/queues/Slide1.png)

Notice that `back` is initialized to the value `-1` because the enqueue operation
moves `back` to the right first before setting `arr[back]` to the newly enqueued element.

After one enqueue operation the array looks something like:

![After 1 enqueue operation](../resources/images/queues/Slide2.png)

Notice that the `back` index has moved to the right one position (its value increased by 1) and
`front` is unchanged.

After a second enqueue operation the array looks something like:

![After 2 enqueue operations](../resources/images/queues/Slide3.png)

Again, `back` has moved to the right one position and `front` is unchanged.

After a third enqueue operation the array looks something like:

![After 3 enqueue operations](../resources/images/queues/Slide4.png)

After a fifth enqueue operation the array looks something like:

![After 5 enqueue operations](../resources/images/queues/Slide5.png)

When a element is dequeued, the following steps occur:

1. the array element at index `front` is saved in a temporary variable so that it can be returned to the caller,
2. the array element `arr[front]` is nulled out so that a reference to the dequeued object is not left in the array,
3. `front` moves one position to the right (its value increased by 1), and
4. the dequeued element is returned to the caller

After these steps occur the array looks something like:

![After 1 dequeue operation](../resources/images/queues/Slide6.png)

Notice that `front` has moved to the right one position and `back` is unchanged.

Dequeueing a second element causes the array to look something like:

![After 2 dequeue operations](../resources/images/queues/Slide7.png)

Again, `front` has moved to the right one position and `back` is unchanged.


Dequeueing two more elements leaves only one element in the queue and causes the array to look something like:

![After 4 dequeue operations](../resources/images/queues/Slide8.png)

Dequeueing the last element from the queue causes `front` to move one position to the right of `back` which suggests
(mistakenly) that this is one way to determine if the queue is empty:

![After 5 dequeue operations](../resources/images/queues/Slide9.png)

Enqueueing an element again causes `back` to move one position to the right:

![After 1 more enqueue operations](../resources/images/queues/Slide10.png)

If we enqueue ten more elements then the `back` index reaches the last valid index of the array;
however the queue is not full as there is still spare capacity at the front of the array:

![After 8 more enqueue operations](../resources/images/queues/Slide11.png)

Enqueueing one more element causes the `back` index to wrap around to the front of the 
array:

![Wrapping of the `back` index](../resources/images/queues/Slide12.png)

There is nothing special about the fact that the `back` index has wrapped around to the 
front of the array except that we have to be careful when we compute the value of `back`
(see the discussion in the next cell).

Enqueueing one more element causes `back` to move one position to the right:

![After 1 more enqueue operation](../resources/images/queues/Slide13.png)

If we enqueue four elements the queue becomes full:

![A full queue](../resources/images/queues/Slide14.png)

Notice that `front` is one position
to the right of `back` just like in an empty queue; this is why we cannot determine if the queue is empty
simply by examining the values of `front` and `back`.

Like the `back` index, the `front` index can also wrap around. Dequeueing ten elements
causes `front` to be equal to the last valid index of the array:

![After 10 more dequeue operations](../resources/images/queues/Slide15.png)

And then dequeueing one more element causes `front` to wrap around the beginning
of the array:

![After 1 more dequeue operation](../resources/images/queues/Slide16.png)

The array combined with the fields `front` and `back` form a data structure called a
[circular buffer](https://en.wikipedia.org/wiki/Circular_buffer). A circular buffer
can be thought of as an array where the two ends of the array are connected to
one another to form a circular arrangement of elements. The array shown in the previous
image above can be drawn as a circular buffer like so:

![Circular buffer](../resources/images/queues/Slide17.png)

When viewed as a circular buffer, `front` and `back` always move in the clockwise
direction.

**Exercise 1** In a non-empty queue, what class invariants exist regarding the fields `front` and `back`?

**Exercise 2** In a non-empty queue, can we calculate the value of `size` using the values of `front` and `back`?

### Computing `front` and `back`

It should be clear to the reader that the values `front` and `back` are both circular,
or cyclic, quantities. Recall from the [Arithmetic notebook](../part1/arithmetic.ipynb#remainder)
that the remainder operator (`%`) is often used when working with cyclic quantities. When shifting
`front` one position to the right in the array (clockwise in the circular buffer) we can compute the
appropriate value like so:

```java
this.front = (this.front + 1) % this.arr.length;
```

`back` can be updated in a similar way by changing both occurrences of `this.front` to `this.back`.

## A fixed-capacity array-based queue

An implementation of the fixed-capacity array-based queue described above is shown in the following cell:

In [ ]:
%classpath add jar ../resources/jar/notes.jar
import java.util.Arrays;
import ca.queensu.cs.cisc124.notes.generics.basics.Queue;
import ca.queensu.cs.cisc124.notes.generics.basics.Stack;
import ca.queensu.cs.cisc124.notes.generics.basics.ArrayStack;



public class ArrayQueue<E> implements Queue<E> {

    // the default capacity of the queue
    private static final int DEFAULT_CAPACITY = 16;

    // the array that stores the queue
    private Object[] arr;

    // the index of the element at the front of the queue
    private int front;
    
    // the index of the element at the back of the queue
    private int back;

    // the number of elements in the queue
    private int size;
    
    
    public ArrayQueue() {
        this.arr = new Object[ArrayQueue.DEFAULT_CAPACITY];
        this.front = 0;
        this.back = -1;
        this.size = 0;
    }
    
    
    public ArrayQueue(ArrayQueue<E> q) {
        this.arr = Arrays.copyOf(q.arr, q.arr.length);
        this.front = q.front;
        this.back = q.back;
        this.size = q.size;
    }
    
    
    @Override
    public int size() {
        return this.size;
    }


    @Override
    public void enqueue(E elem) {
        if (this.size == this.arr.length) {
            throw new RuntimeException("size == capacity");
        }
        this.back = (this.back + 1) % this.arr.length;
        this.arr[this.back] = elem;
        this.size++;
    }

    
    @Override
    public E dequeue() {
        if (this.isEmpty()) {
            throw new RuntimeException("dequeued an empty queue");
        }
        Object elem = this.arr[this.front];
        
        // null out the value stored in the array
        this.arr[this.front] = null;
        this.front = (this.front + 1) % this.arr.length;
        this.size--;
        return (E) elem;
    }

    
    @Override
    public E front() {
        if (this.isEmpty()) {
            throw new RuntimeException("no front element in an empty queue");
        } 
        return (E) this.arr[this.front];
    }

    
    @Override
    public E back() {
        if (this.isEmpty()) {
            throw new RuntimeException("no back element in an empty queue");
        }
        return (E) this.arr[this.back];
    }
    
    
    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("[");
        for (int i = 0; i < this.size - 1; i++) {
            int index = (this.front + i) % this.arr.length;
            b.append(this.arr[index].toString());
            b.append(", ");
        }
        if (!this.isEmpty()) {
            b.append(this.back());
        }
        b.append("]");
        
        return b.toString();
    }
}


The next cell contains a small program that performs all of the steps in the examples shown above:

In [ ]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.generics.basics.Queue;

ArrayQueue<String> q = new ArrayQueue<>();
q.enqueue("A");
q.enqueue("B");
q.enqueue("C");
q.enqueue("D");
q.enqueue("E");
System.out.println("q: " + q);
System.out.println();

String got = q.dequeue();
System.out.println("dequeued: " + got + ", q: " + q);
System.out.println();

got = q.dequeue();
System.out.println("dequeued: " + got + ", q: " + q);
System.out.println();

got = q.dequeue();
System.out.println("dequeued: " + got + ", q: " + q);
System.out.println();

got = q.dequeue();
System.out.println("dequeued: " + got + ", q: " + q);
System.out.println();

got = q.dequeue();
System.out.println("dequeued: " + got + ", q: " + q);
System.out.println();

q.enqueue("F");
q.enqueue("G");
q.enqueue("H");
q.enqueue("I");
q.enqueue("J");
q.enqueue("K");
q.enqueue("L");
q.enqueue("M");
q.enqueue("N");
q.enqueue("O");
q.enqueue("P");
System.out.println("q: " + q);
System.out.println();

q.enqueue("Q");
q.enqueue("R");
q.enqueue("S");
q.enqueue("T");
q.enqueue("U");
System.out.println("q: " + q);
System.out.println();

for (int i = 0; i < 10; i++) {
    got = q.dequeue();
    System.out.println("dequeued: " + got + ", q: " + q);
}
System.out.println();

got = q.dequeue();
System.out.println("dequeued: " + got + ", q: " + q);


## Resizing the array

We need to reallocate the array when an element is enqueued into a full queue. Suppose that we use the same approach that we used when
implementing the `push` method for an array-based stack: Use `Arrays.copyOf` to create a copy of the array having larger capacity:


```java 
    @Override
    public void enqueue(E elem) {
        if (this.size == this.arr.length) {
            this.arr = Arrays.copyOf(this.arr, this.arr.length * 2);    // oops
        }
        this.back = (this.back + 1) % this.arr.length;
        this.arr[this.back] = elem;
        this.size++;
    }
```

This results in an array of greater capacity but we no longer have a circular buffer as shown in the following figure:

![Naive resize and copy](../resources/images/queues/naive-resize.png)

Notice that there is no free space immediately after `back` in the array.

The correct implementation of `enqueue` must unwind the circular buffer by copying the elements of the original array from `front` to `back` into the new array, and
then updating `front` and `back` to their correct values in the new array:

![](../resources/images/queues/correct-resize.png)

**Exercise 3** Implement an `enqueue` operation that resizes the circular buffer when required as shown above.

**Exercise 4** Is there another way to unwind the circular buffer other than the one shown above? If so, describe how to do so.

**Exercise 5** What is the worst-case computational complexity of `dequeue` in an array-based queue?

**Exercise 6** What is the worst-case computational complexity of `enqueue` in an array-based queue?

**Exercise 7** Add a method to the `ArrayQueue` class that reverses the order of the elements in this queue. You may use a second temporary queue or stack to help you.

**Exercise 8** Repeat Exercise 7 without using a second temporary data structure. This is a bit tricky to get correct; students should attempt this exercise before looking at the solution.

In [ ]:
// Exercises 7 and 8 with imports added for you

%classpath add jar ../resources/jar/notes.jar
import java.util.Arrays;
import ca.queensu.cs.cisc124.notes.generics.basics.ArrayStack;
import ca.queensu.cs.cisc124.notes.generics.basics.Queue;
import ca.queensu.cs.cisc124.notes.generics.basics.Stack;

public class ArrayQueue<E> implements Queue<E> {

    // the default capacity of the queue
    private static final int DEFAULT_CAPACITY = 16;

    // the array that stores the queue
    private Object[] arr;

    // the index of the element at the front of the queue
    private int front;
    
    // the index of the element at the back of the queue
    private int back;

    // the number of elements in the queue
    private int size;
    
    
    public ArrayQueue() {
        this.arr = new Object[ArrayQueue.DEFAULT_CAPACITY];
        this.front = 0;
        this.back = -1;
        this.size = 0;
    }
    
    
    public ArrayQueue(ArrayQueue<E> q) {
        this.arr = Arrays.copyOf(q.arr, q.arr.length);
        this.front = q.front;
        this.back = q.back;
        this.size = q.size;
    }
    
    
    @Override
    public int size() {
        return this.size;
    }


    @Override
    public void enqueue(E elem) {
        if (this.size == this.arr.length) {
            throw new RuntimeException("size == capacity");
        }
        this.back = (this.back + 1) % this.arr.length;
        this.arr[this.back] = elem;
        this.size++;
    }

    
    @Override
    public E dequeue() {
        if (this.isEmpty()) {
            throw new RuntimeException("dequeued an empty queue");
        }
        Object elem = this.arr[this.front];
        
        // null out the value stored in the array
        this.arr[this.front] = null;
        this.front = (this.front + 1) % this.arr.length;
        this.size--;
        return (E) elem;
    }

    
    @Override
    public E front() {
        if (this.isEmpty()) {
            throw new RuntimeException("no front element in an empty queue");
        } 
        return (E) this.arr[this.front];
    }

    
    @Override
    public E back() {
        if (this.isEmpty()) {
            throw new RuntimeException("no back element in an empty queue");
        }
        return (E) this.arr[this.back];
    }
    
    
    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("[");
        for (int i = 0; i < this.size - 1; i++) {
            int index = (this.front + i) % this.arr.length;
            b.append(this.arr[index].toString());
            b.append(", ");
        }
        if (!this.isEmpty()) {
            b.append(this.back());
        }
        b.append("]");
        
        return b.toString();
    }
}